# Preparation of the datasets for BERT-based models fine-tuning

## Instalando pacotes

In [ ]:
!pip install -U --quiet transformers datasets spacy

In [ ]:
!python -m spacy download 'pt_core_news_lg'

## Importando pacotes

In [ ]:
import pandas as pd
import os
import json

from datasets.dataset_dict import DatasetDict
from datasets import load_from_disk
from datasets import Dataset

from transformers import AutoTokenizer
from ast import literal_eval

import spacy
spacy_model = "pt_core_news_lg"
nlp = spacy.load(spacy_model)

## Carregando Dados

In [ ]:
root_folder = '.'
data_folder = f"{root_folder}/datasets/"

# Bertimbau
model_name     = "neuralmind/bert-base-portuguese-cased"
model_nickname = "bertimbau"

# mBERT
#model_name     = "bert-base-multilingual-cased"
#model_nickname = "mbert"

model_folder = f"{root_folder}/{model_nickname}"

if not os.path.exists(model_folder):
    os.mkdir(model_folder)

print(model_name)
print(model_folder)

In [ ]:
# carregando dados anotados e convertendo a coluna do ner para lista
train = pd.read_csv(f"{data_folder}train.csv", sep=';', keep_default_na=False, converters={"ner": literal_eval})
val   = pd.read_csv(f"{data_folder}val.csv",   sep=';', keep_default_na=False, converters={"ner": literal_eval})
test  = pd.read_csv(f"{data_folder}test.csv",  sep=';', keep_default_na=False, converters={"ner": literal_eval})

print(f"Train:{len(train)}, Val:{len(val)}, Test:{len(test)}")
print(len(train)+len(val)+len(test))

In [ ]:
train

## Preparando dados para treinamento/validacao/teste do Modelo


In [ ]:
def keys_to_int(x):
    return {int(k): v for k, v in x.items()}

int2label = []
with open(f"{root_folder}/labels.json", "r") as f:
    int2label = json.load(f, object_hook=keys_to_int)

label2int = {label: index for index, label in int2label.items()}
label2int

print(int2label)
print(label2int)

In [ ]:
def get_dados(dados: pd.DataFrame):
    ds = {}
    ds['text'] = []
    ds['tokens'] = []
    ds['ner_tags'] = []
    for idx in range(0, len(dados)):

        text = dados.loc[idx, 'text']
        ner  = dados.loc[idx, 'ner']

        tokens     = [ t.text for t in nlp(text) ] 
        ner_tags   = [0] * len(tokens)
        
        pos = 0
        for item in ner:

            w_tokens = [ t.text for t in nlp(item[2]) ] # entidade
            label    = item[3] # label
            w_tokens_len = len(w_tokens)

            achou = False
            while pos < len(tokens) and not achou:

                print(f"{tokens[ pos : pos + w_tokens_len]} -> {w_tokens}")
                if tokens[ pos : pos + w_tokens_len ] == w_tokens:

                    ner_tags[pos] = label2int["B-"+label]
                    for p_i in range(pos+1, pos + w_tokens_len):
                       ner_tags[p_i] = label2int["I-"+label]

                    pos = pos + w_tokens_len
                    achou = True

                else:
                    pos += 1

        ds['text'].append(text)
        ds['tokens'].append(tokens)
        ds['ner_tags'].append(ner_tags)

    return ds


#df_test = pd.DataFrame()

# sentenças do conjunto de Test
#4
#df_test['text'] = ["Bom, nessa saga, nesse processo contínuo de melhoria da experiência dos nossos clientes a gente vem investindo fortemente na questão do digital por meio da aceleração do processo, da transformação digital do Banco do Brasil."]
#df_test['ner'] = "[[79, 87, 'clientes', 'CLIENTE'], [208, 223, 'Banco do Brasil', 'COMPANY']]"

#84
#df_test['text'] = ["Nós já solicitamos, temos autorização do Banco Central para fazer um novo processo de recompra, só que o bond do Banco está sendo negociado em torno de 104% do seu valor de fato."]
#df_test['ner'] = "[[41, 54, 'Banco Central', 'ORG'], [113, 118, 'Banco', 'COMPANY'], [152, 156, '104%', 'PERCENTUAL']]"

#122
#df_test['text'] = ["Senhor João Luiz: Pegando a respeito dessa linha, essa linha cresceu 254 milhões, que aliás dá exatamente 254% também, eu queria saber exatamente isso, quanto é desse reforço de provisão para essas receitas contábil que você disse e, enfim, que foi feito aquele estoque que você disse de 1.6 bi, ou seja, isso migrando para, eventualmente, migrando para a Previ, que já está adiantado, essa linha pararia de crescer e qual o tamanho dela?"]
#df_test['ner'] = "[[106, 110, '254%', 'PERCENTUAL'], [178, 186, 'provisão', 'PROVISAO']]"

#228
#df_test['text'] = ["Na página 10, com relação à nossa carteira de crédito pessoal, nós temos três produtos: o nosso crédito na conta, que é o nosso crédito emergencial, a antecipação de FGTS e o nosso cartão de crédito aberto."]
#df_test['ner'] = "[[34, 53, 'carteira de crédito', 'CARTEIRA'], [181, 198, 'cartão de crédito', 'PRODUTO']]"

#237
#df_test['text'] = ["Quer dizer, houve um crescimento de 18,5%, muito acima da taxa de crescimento da receita de serviços do Banco."]
#df_test['ner'] = "[[36, 41, '18,5%', 'PERCENTUAL'], [81, 100, 'receita de serviços', 'RECEITA'], [104, 109, 'Banco', 'COMPANY']]"

#284
#df_test['text'] = ["As operações com atraso de mais de 180 dias sempre estiveram abaixo de 2% da carteira de crédito e a relação entre provisão e carteira gira em torno não extrapolou os 3,5%, chegando a 3,1% em setembro de 2012."]
#df_test['ner'] = "[[71, 73, '2%', 'PERCENTUAL'], [77, 96, 'carteira de crédito', 'CARTEIRA'], [115, 123, 'provisão', 'PROVISAO'], [126, 134, 'carteira', 'CARTEIRA'], [167, 171, '3,5%', 'PERCENTUAL'], [184, 188, '3,1%', 'PERCENTUAL']]"

#392
#df_test['text'] = ["Em princípio, esse instrumento de 8,1 bilhões que negociamos com o Ministério da Fazenda e depois com autorização do Banco Central do Brasil para compor o capital de nível 1 do Banco, em princípio foi um movimento que fizemos mas não existe nenhum indicativo de que venhamos a fazer qualquer coisa dessa natureza da forma como fizemos no passado."]
#df_test['ner'] = "[[117, 140, 'Banco Central do Brasil', 'ORG'], [177, 182, 'Banco', 'COMPANY']]"

#396
#df_test['text'] = ["A Despesa de Provisão para Devedores Duvidosos (PDD) atingiu 46,5 milhões de Reais no trimestre – um aumento de 35,1% em relação ao trimestre anterior, e uma queda de 50,1% em relação ao mesmo período de 2020."]
#df_test['ner'] = "[[2, 9, 'Despesa', 'DESPESA'], [13, 21, 'Provisão', 'PROVISAO'], [48, 51, 'PDD', 'PROVISAO'], [112, 117, '35,1%', 'PERCENTUAL'], [167, 172, '50,1%', 'PERCENTUAL']]"

#721
#df_test['text'] = ["Mudando agora para o slide número 12, falando um pouco da gestão de risco, começando pelo risco de liquidez."]
#df_test['ner'] = "[[68, 73, 'risco', 'RISCO'], [90, 107, 'risco de liquidez', 'INDICADOR_LIQUIDEZ']]"

#751
#df_test['text'] = ["A primeira sobre a despesa de PDD, a gente viu que ela foi bem elevada neste trimestre, vocês acreditam que ela deve voltar próximo à média de R$ 20 milhões/trimestre ou ela provavelmente ela vai se manter bem elevada próximo a esse patamar de R$ 40 milhões devido ao cenário macro mais desafiador."]
#df_test['ner'] = "[[19, 26, 'despesa', 'DESPESA'], [30, 33, 'PDD', 'PROVISAO'], [143, 156, 'R$ 20 milhões', 'MONEY'], [244, 257, 'R$ 40 milhões', 'MONEY']]"

#846
#df_test['text'] = ["Essa despesa de provisão ela está ligada ao fato de que a carteira de crédito imobiliário do Banco sofreu uma retração no trimestre, e provocou, na parte do guidance um ajuste para efeito de performance."]
#df_test['ner'] = "[[5, 12, 'despesa', 'DESPESA'], [16, 24, 'provisão', 'PROVISAO'], [58, 77, 'carteira de crédito', 'CARTEIRA'], [93, 98, 'Banco', 'COMPANY']]"

#1002
#df_test['text'] = ["Então, todo efeito de alta de juros tem muito mais efeito na margem com clientes da forma como a gente pública do que na margem com o mercado, tá."]
#df_test['ner'] = "[[22, 35, 'alta de juros', 'CONDICOES_MACROECONOMICAS'], [72, 80, 'clientes', 'CLIENTE']]"

#1966
#df_test['text'] = ["A estrutura de capital total do Banco, 17,7%, sendo o capital de nível 1 14%, extremamente robusto."]
#df_test['ner'] = "[[32, 37, 'Banco', 'COMPANY'], [39, 44, '17,7%', 'PERCENTUAL'], [73, 76, '14%', 'PERCENTUAL']]"

#2022
#df_test['text'] = ["Vale mencionar que nós vamos continuar crescendo a carteira de crédito, crescendo de forma robusta, mas mantendo toda a filosofia em termos de carteira de crédito."]
#df_test['ner'] = "[[51, 70, 'carteira de crédito', 'CARTEIRA'], [143, 162, 'carteira de crédito', 'CARTEIRA']]"


#df_test['text'] = ["Bom, nessa saga, nesse processo contínuo de melhoria da experiência dos nossos clientes a gente vem investindo fortemente na questão do digital por meio da aceleração do processo, da transformação digital do Banco do Brasil."]
#df_test['ner'] = "[[79, 87, 'clientes', 'CLIENTE'], [208, 223, 'Banco do Brasil', 'COMPANY']]"

#df_test['text'] = ["O lucro líquido do Santander aumentou"]
#df_test['ner'] = "[[2, 15, 'lucro líquido', 'LUCRO'], [19, 28, 'Santander', 'COMPANY']]"

#df_test['ner'] = df_test['ner'].apply(literal_eval)
#result = get_dados(df_test)
#for key in result.keys():
#  print(f"{len(result[key][0])} => {result[key]}")

In [ ]:
dados_train = get_dados(train)
dados_val   = get_dados(val)
dados_test  = get_dados(test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples, batch_size = 30000):
    all_tokenized_samples = {}
    all_tokenized_samples["labels"] = []
    all_tokenized_samples["texts"]  = []
    all_tokenized_samples["tokens"] = []
    print(f"all samples len: {len(all_samples['text'])}")
    for i in range(0, len(all_samples["text"]), batch_size):

        samples_text   = all_samples['text'][i : i + batch_size]
        samples_tokens = all_samples['tokens'][i : i + batch_size]
        samples_ner    = all_samples['ner_tags'][i : i + batch_size]
        tokenized_samples = tokenizer.batch_encode_plus(
            samples_tokens
            , max_length=512
            , truncation=True
            , is_split_into_words=True)

        total_adjusted_labels = []

        for k in range(0, len(tokenized_samples["input_ids"])):
            prev_wid = -1
            word_ids_list = tokenized_samples.word_ids(batch_index=k)

            existing_label_ids = samples_ner[k]

            i = -1
            adjusted_label_ids = []

            for wid in word_ids_list:
                if (wid is None):
                    adjusted_label_ids.append(-100)
                elif (wid != prev_wid):
                    i += 1
                    adjusted_label_ids.append(existing_label_ids[i])
                    prev_wid = wid
                else:
                    adjusted_label_ids.append(existing_label_ids[i])

            total_adjusted_labels.append(adjusted_label_ids)

        for key in tokenized_samples.keys():
            if key not in all_tokenized_samples.keys():
                all_tokenized_samples[key] = []

            all_tokenized_samples[key].extend(tokenized_samples[key])
            #print(f"Tokenized len: {len(tokenized_samples[key])}, All token.: {len(all_tokenized_samples[key])}")

        all_tokenized_samples["labels"].extend(total_adjusted_labels)
        all_tokenized_samples["texts"].extend(samples_text)
        all_tokenized_samples["tokens"].extend(samples_tokens)

    return all_tokenized_samples


print("Tokenizando conjunto de treino...")
tokenized_train = tokenize_adjust_labels(dados_train)

print("Tokenizando conjunto de validação...")
tokenized_val   = tokenize_adjust_labels(dados_val)

print("Tokenizando conjunto de teste...")
tokenized_test  = tokenize_adjust_labels(dados_test)


## Salvando dados processados

In [ ]:
d = {'train':Dataset.from_dict({'texts'         : tokenized_train['texts'],
                                'tokens'        : tokenized_train['tokens'],
                                'input_ids'     : tokenized_train['input_ids'],
                                'attention_mask': tokenized_train['attention_mask'],
                                'labels'        : tokenized_train['labels']}),
     'validation':Dataset.from_dict({'texts'         : tokenized_val['texts'],
                                     'tokens'        : tokenized_val['tokens'],
                                     'input_ids'     : tokenized_val['input_ids'],
                                     'attention_mask': tokenized_val['attention_mask'],
                                     'labels'        : tokenized_val['labels']}),
     'test':Dataset.from_dict({'texts'         : tokenized_test['texts'],
                               'tokens'        : tokenized_test['tokens'],
                               'input_ids'     : tokenized_test['input_ids'],
                               'attention_mask': tokenized_test['attention_mask'],
                               'labels'        : tokenized_test['labels']})
     }

dataset = DatasetDict(d)
dataset

In [ ]:
# salvando dados processados em arquivos
dataset.save_to_disk(f"{model_folder}/dataset-bancos-{model_nickname}")

## Analisando dados

In [ ]:
dataset = load_from_disk(f"{model_folder}/dataset-bancos-{model_nickname}")
dataset

In [ ]:
# calculando numero de tokens gerados com o Spacy, numero de subwords gerados pelo tokenizer e a proporção entre eles
rows = []
for conjunto_nome in dataset.keys():
    conjunto_tokens     = dataset[conjunto_nome]['tokens']
    conjunto_input_ids  = dataset[conjunto_nome]['input_ids']
    len_conjunto = len(conjunto_tokens)
    for i in range(len_conjunto):
        print(f"\r{conjunto_nome}: {i}/{len_conjunto}", end="")
        num_tokens    = len(conjunto_tokens[i])
        num_subwords  = len(conjunto_input_ids[i]) - 2 # menos dois tokens especiais CLS e SEP
        proporcao     = num_subwords / num_tokens
        row = {'conjunto': conjunto_nome
               , "num_tokens": num_tokens
               , "num_subwords": num_subwords
               , "proporcao": proporcao
               }

        rows.append(row)
    print()

df_metrics = pd.DataFrame.from_dict(rows)
df_metrics

In [ ]:
df_metrics.to_csv(f"{model_folder}/proporcao-tokens-{model_nickname}.csv", sep=';', index=False)

In [ ]:
df_metrics = pd.read_csv(f"{model_folder}/proporcao-tokens-{model_nickname}.csv", sep=';')

In [ ]:
print(df_metrics['num_tokens'].sum())
print(df_metrics['num_subwords'].sum())


In [ ]:
df_metrics['num_subwords'].sum() / df_metrics['num_tokens'].sum()

In [ ]:
df_metrics['num_subwords'].max()

In [ ]:
len(tokenizer.get_vocab())